In [2]:
import tensorflow as pb

In [4]:
import onnxruntime as rt

In [5]:
import numpy as np

In [6]:
random_input = np.random.rand(1,3,224,224)

In [9]:
random_input = random_input.astype(np.float32)

In [11]:
sess = rt.InferenceSession("/Users/bob/code/ByteDance/PNN/resource/models/converter/mobilenetv2-7.onnx")

In [17]:
input_name = sess.get_inputs()[0].name

In [18]:
input_name

'input'

### RUN ONNX

In [19]:
res = sess.run(None, {input_name : random_input})

In [ ]:
res

In [23]:
pnn_input = np.transpose(random_input, (0,2,3,1))

In [27]:
pnn_input.tofile("/Users/bob/code/ByteDance/PNN/resource/models/onnx_input.bin")

In [ ]:
res

In [49]:
pnn_output = np.transpose(res[0], (1,0))

In [50]:
res[0].shape

(1, 1000)

In [34]:
res[0].tofile("/Users/bob/code/ByteDance/PNN/tools/converter/test/test_data/onnx/onnx_output.bin")

In [35]:
import torch

In [41]:
conv = torch.nn.Conv2d(3,32,3,2,1)

In [42]:
conv(torch.tensor(random_input)).shape

torch.Size([1, 32, 112, 112])

In [43]:
random_input.shape

(1, 3, 224, 224)

### Compare Weight

In [64]:
import onnx
from onnx import numpy_helper

In [65]:
m = onnx.load("/Users/bob/code/ByteDance/PNN/resource/models/converter/mobilenetv2-7.onnx")

In [66]:
INTIALIZERS = m.graph.initializer

In [67]:
onnx_weights = {}
for initializer in INTIALIZERS:
    W = numpy_helper.to_array(initializer)
    onnx_weights[initializer.name] = W

In [93]:
onnx_weights['475'].flatten()[:100]

array([ 1.8972903e-03, -6.2180799e-04,  2.1329592e-03,  4.7169146e-03,
       -3.6527440e-03,  9.8670297e-04,  1.5179312e-03, -5.3740530e-03,
       -2.1191251e-03,  1.1499577e-03, -8.5107912e-04,  2.1692887e-03,
        2.8776766e-03, -4.7288486e-03, -3.0014609e-04,  1.6332326e-03,
       -4.7421642e-03, -1.1329182e-03, -3.6309685e-03, -2.9019027e-03,
       -1.4334710e-03, -1.6134927e-03, -4.2111590e-03, -2.1897089e-03,
       -3.8176179e-03, -4.8130783e-03, -3.4843918e-03, -2.7336318e-02,
       -9.1415131e-03,  9.1914237e-03,  5.5952542e-02,  1.6294515e-01,
        1.8018376e-02, -8.0655001e-02, -1.1983158e-01, -1.6452458e-02,
       -2.8886281e-02,  2.2808637e-02,  7.8767102e-04,  1.1442577e-01,
        3.2129836e-01,  4.3629289e-02, -1.4437810e-01, -2.6112846e-01,
       -3.2973059e-02, -6.7799860e-03, -4.6871006e-03,  1.1295444e-02,
        1.8052191e-02,  7.4299596e-02,  6.6330755e-04, -2.6663985e-02,
       -6.7156382e-02,  4.4175959e-04, -2.2961097e-02,  2.1333840e-02,
      

In [99]:
np.transpose(onnx_weights['475'],(0,2,3,1)).flatten()[:100]

array([ 1.89729035e-03,  1.14995765e-03, -3.63096851e-03, -6.21807994e-04,
       -8.51079123e-04, -2.90190266e-03,  2.13295920e-03,  2.16928869e-03,
       -1.43347098e-03,  4.71691461e-03,  2.87767663e-03, -1.61349273e-03,
       -3.65274400e-03, -4.72884858e-03, -4.21115896e-03,  9.86702973e-04,
       -3.00146086e-04, -2.18970887e-03,  1.51793123e-03,  1.63323258e-03,
       -3.81761789e-03, -5.37405303e-03, -4.74216416e-03, -4.81307833e-03,
       -2.11912510e-03, -1.13291817e-03, -3.48439184e-03, -2.73363180e-02,
       -2.88862810e-02, -6.77998597e-03, -9.14151315e-03,  2.28086375e-02,
       -4.68710065e-03,  9.19142365e-03,  7.87671015e-04,  1.12954443e-02,
        5.59525415e-02,  1.14425771e-01,  1.80521905e-02,  1.62945151e-01,
        3.21298361e-01,  7.42995963e-02,  1.80183761e-02,  4.36292887e-02,
        6.63307554e-04, -8.06550011e-02, -1.44378096e-01, -2.66639851e-02,
       -1.19831577e-01, -2.61128455e-01, -6.71563819e-02, -1.64524578e-02,
       -3.29730585e-02,  

In [76]:
onnx_weights['475'][0,:,:,:] * random_input[0,:,0:3,0:3] + onnx_weights['476'][0]

array([[[-0.08438151, -0.08606348, -0.08446034],
        [-0.08552954, -0.0887817 , -0.08574593],
        [-0.08553703, -0.08831108, -0.08750756]],

       [[-0.08525221, -0.08611091, -0.08575991],
        [-0.08334363, -0.0880964 , -0.08599799],
        [-0.08572925, -0.0883361 , -0.08642525]],

       [[-0.08720399, -0.0879343 , -0.08672588],
        [-0.08606537, -0.08625247, -0.08712777],
        [-0.08693956, -0.08886247, -0.08880404]]], dtype=float32)

In [81]:
(onnx_weights['475'][0,:,:,:] * random_input[0,:,0:3,0:3]).sum()+ onnx_weights['476'][0]

-0.101355925

#### depthwise卷积477

In [97]:
onnx_weights['481'].flatten()[:100]

array([-1.05721653e-02,  6.78157210e-02, -7.16115907e-02,  1.80039734e-01,
       -1.75979491e-02,  7.50672936e-01,  2.39875332e-01, -2.36762576e-02,
       -5.02474867e-02,  5.64223016e-03, -2.92116124e-03,  2.80231982e-01,
        8.27982649e-02, -1.21604361e-01, -1.89991342e-03, -4.95574577e-03,
       -4.36248165e-03,  2.36865436e-03,  4.73081231e-01, -3.51491183e-01,
       -2.11996526e-01, -4.76436270e-03, -8.77841413e-02, -5.17852418e-03,
        1.68323591e-01,  2.84582406e-01,  7.24008605e-02,  3.79971862e-02,
        3.76426764e-02,  8.98576975e-02,  2.95643419e-01,  1.94637969e-01,
        2.39149798e-02,  2.37236936e-02, -7.98205793e-01, -1.34331435e-01,
        1.39313534e-01, -2.16891170e-02, -8.12713802e-02,  1.19389869e-01,
        1.50937065e-01, -1.84907389e-04,  1.94397420e-01, -4.72135991e-02,
       -1.70496613e-01,  2.85360098e-01,  1.41558400e-03,  2.22358643e-03,
        4.89142025e-03, -2.73317588e-03,  1.82510644e-01,  5.85021436e-01,
        2.99954891e-01, -

In [96]:
np.transpose(onnx_weights['478'],(1,2,3,0)).flatten()[:100]

array([-7.40940943e-02,  4.65403050e-02,  1.36745851e-02, -2.69007266e-01,
        1.41356874e+00, -1.13077842e-01, -2.20794827e-02, -8.28864396e-01,
       -1.66269600e-01, -6.39762590e-03,  3.89419794e-01,  5.09818435e-01,
       -3.86324599e-02, -7.71382034e-01, -4.00574587e-04,  2.09500990e-03,
       -3.53145301e-02, -7.09671108e-03,  1.56361079e+00,  3.25368881e-01,
       -2.15556949e-01,  6.19352572e-02,  1.91454455e-01, -4.84627439e-03,
        1.18434548e-01, -3.13234359e-01, -1.09805286e-01,  7.09305406e-02,
        1.15237422e-01,  8.42426866e-02,  1.67386398e-01,  8.34350884e-01,
       -8.87888074e-02, -1.34104609e-01, -1.19427428e-01, -2.02732980e-01,
        4.60554409e+00,  3.81569028e-01,  6.07096732e-01, -4.22798872e-01,
        6.31065249e-01, -1.80347031e-03, -4.12419617e-01,  4.32233047e+00,
        1.20860562e-01, -2.94338800e-02, -1.02723239e-03, -6.20504841e-03,
       -2.69466005e-02,  8.87986564e-04, -6.08361101e+00, -4.45463240e-01,
        4.09713164e-02, -

### Compare Conv via torch

In [83]:
import torch.nn.functional as F

In [86]:
a = F.conv2d(torch.tensor(random_input),torch.tensor(onnx_weights['475']),torch.tensor(onnx_weights['476']),2,1)

In [89]:
a.flatten()[:10]

tensor([-0.1024, -0.0980, -0.1046, -0.1007, -0.0979, -0.0966, -0.0932, -0.0989,
        -0.1028, -0.0976])

### DeepFM test

In [106]:
d_sess = sess = rt.InferenceSession("/Users/bob/code/ByteDance/PNN/resource/models/converter/deepfm.onnx")

In [108]:
rtt1 = np.array((4.0,),dtype=np.float32)
rtt2 = np.array((9.0,),dtype=np.float32)
rtt3 = np.array((2.0,),dtype=np.float32)
rtt4 = np.array((3.0,),dtype=np.float32)
rtt5 = np.array((7.0,),dtype=np.float32)

In [126]:
timeGap1 = np.array((3,),dtype=np.float32)
timeGap2 = np.array((5,),dtype=np.float32)
timeGap3 = np.array((7,),dtype=np.float32)
timeGap4 = np.array((4,),dtype=np.float32)
timeGap5 = np.array((7,),dtype=np.float32)

In [127]:
net = np.array((1,),dtype=np.int32)

In [128]:
so = np.array((1,),dtype=np.int32)

In [129]:
retry = np.array((1,),dtype=np.int32)

In [130]:
nc = np.array((1,),dtype=np.int32)

In [131]:
i_m = {"rtt1" : rtt1,
      "rtt2" : rtt2,
                                           "rtt3" : rtt3,           "rtt4" : rtt4,
                                           "rtt5" : rtt5,           "time_gap_1" : timeGap1,
                                           "time_gap_2" : timeGap2,     "time_gap_3" : timeGap3,
                                           "time_gap_4" : timeGap4,     "time_gap_5" : timeGap5,
                                           "network_type" : net,   "socket_reused" : so,
                                           "retry_attempts" : retry, "networktype_changed" : nc}           

In [132]:
d_res = sess.run(None, i_m)

In [133]:
d_res

[array([[0.03631085, 0.51652163, 0.44716746]], dtype=float32),
 array([1], dtype=int64)]

In [2]:
import tensorflow as tf

In [26]:
a = tf.constant([[[[2,3,4],[2,3,5]]]])

In [27]:
a

<tf.Tensor: shape=(1, 1, 2, 3), dtype=int32, numpy=
array([[[[2, 3, 4],
         [2, 3, 5]]]], dtype=int32)>

In [28]:
tf.strided_slice(a,[0,0,0,0],[2,1,2,3],shrink_axis_mask=3)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 4],
       [2, 3, 5]], dtype=int32)>

In [31]:
a.numpy().flatten()

array([2, 3, 4, 2, 3, 5], dtype=int32)

In [32]:
import numpy as np

In [33]:
np.fromfile("/Users/bob/code/ByteDance/PNN/tools/converter/test/test_data/onnx/mobilenetv2/onnx_random_input.bin",dtype=float)

array([1.66843674e-04, 9.45458847e-11, 2.05066898e-04, ...,
       8.12022512e-06, 1.89962498e-12, 1.93948643e-08])

In [36]:
a = tf.constant(3, shape=(2,6,6,3))

In [48]:
b = tf.random.uniform((2,4,4,3),1, 4,dtype=tf.float32)

In [54]:
tf.reduce_mean(b, axis = (1,2)).numpy().flatten()

array([2.3321366, 2.6642637, 2.333531 , 2.2139106, 2.338769 , 2.6055315],
      dtype=float32)

In [51]:
tf.nn.avg_pool2d(
    b, (4,4), (1,1), "VALID", data_format='NHWC', name=None
)

<tf.Tensor: shape=(2, 1, 1, 3), dtype=float32, numpy=
array([[[[2.3321366, 2.6642637, 2.333531 ]]],


       [[[2.2139106, 2.338769 , 2.6055315]]]], dtype=float32)>

In [53]:
b.numpy().flatten()

array([2.9118795, 3.7052922, 2.3717666, 2.890676 , 1.6145109, 2.9519415,
       1.3341585, 3.9860868, 2.2446895, 3.4455233, 2.7552786, 2.2876463,
       1.9189845, 1.6047716, 1.0437633, 2.1045804, 3.7074776, 2.4548407,
       1.0726353, 1.1634903, 2.2933633, 2.252412 , 1.67491  , 1.986774 ,
       1.5706961, 2.1940758, 3.7276957, 1.6122811, 2.301124 , 1.4659183,
       1.5948964, 3.017662 , 2.6338673, 2.8894773, 3.0096087, 1.0713546,
       2.501935 , 3.1185067, 2.7091215, 2.0054429, 3.528976 , 1.8321829,
       3.568843 , 2.692162 , 3.4607415, 3.6397624, 2.554286 , 2.8008275,
       2.5199256, 3.3145719, 3.8702087, 2.1613386, 3.4517035, 1.7315804,
       2.0326738, 2.0788436, 3.9043756, 1.7780523, 2.4048076, 2.0468464,
       2.8917491, 1.7999939, 2.4324276, 1.8506043, 2.3300376, 3.9369717,
       2.3207133, 1.1431438, 2.0508432, 1.3073529, 2.4114628, 1.841557 ,
       3.9494724, 3.2580204, 2.116208 , 1.6970686, 1.2973379, 2.7349024,
       1.0710549, 3.694911 , 3.3717117, 3.6750503, 

In [60]:
r = tf.random.uniform((2,4,4,3),-2, 10,dtype=tf.float32)

In [61]:
r.numpy().flatten()

array([ 3.7342796 ,  3.793015  ,  6.7969103 ,  5.140199  , -0.741559  ,
        8.965725  ,  4.113853  ,  6.69298   ,  0.8675132 ,  4.0174837 ,
        1.221642  ,  1.352076  ,  9.549547  ,  2.4564223 ,  6.057291  ,
        5.471656  ,  1.4963846 ,  5.005867  ,  2.6941233 ,  1.1625762 ,
        5.262801  ,  2.3044677 , -0.6934724 ,  1.0247207 ,  3.6226711 ,
       -1.6463418 , -0.5841427 ,  3.488874  , -0.0894146 ,  2.950873  ,
        3.3546448 ,  0.25870323,  7.313571  ,  5.002265  ,  7.0031357 ,
        7.495699  ,  0.1534605 , -0.37036848,  2.6488833 ,  3.886014  ,
        3.6057453 ,  3.4239302 , -1.1137309 ,  1.4478288 ,  8.095963  ,
        8.86252   ,  4.031178  , -0.9866142 ,  0.20055723,  3.8126774 ,
        4.8622136 ,  4.0365353 , -1.3078985 ,  8.915903  , -1.1861677 ,
        3.7889724 ,  9.6263485 ,  8.395141  ,  5.8198724 ,  1.9176502 ,
       -0.87991095,  9.989952  , -0.69885254,  3.3308868 , -1.9977984 ,
        9.642432  ,  9.863695  ,  3.0424356 ,  5.390935  ,  8.55

In [64]:
tf.nn.relu6(r).numpy().flatten()

array([3.7342796 , 3.793015  , 6.        , 5.140199  , 0.        ,
       6.        , 4.113853  , 6.        , 0.8675132 , 4.0174837 ,
       1.221642  , 1.352076  , 6.        , 2.4564223 , 6.        ,
       5.471656  , 1.4963846 , 5.005867  , 2.6941233 , 1.1625762 ,
       5.262801  , 2.3044677 , 0.        , 1.0247207 , 3.6226711 ,
       0.        , 0.        , 3.488874  , 0.        , 2.950873  ,
       3.3546448 , 0.25870323, 6.        , 5.002265  , 6.        ,
       6.        , 0.1534605 , 0.        , 2.6488833 , 3.886014  ,
       3.6057453 , 3.4239302 , 0.        , 1.4478288 , 6.        ,
       6.        , 4.031178  , 0.        , 0.20055723, 3.8126774 ,
       4.8622136 , 4.0365353 , 0.        , 6.        , 0.        ,
       3.7889724 , 6.        , 6.        , 5.8198724 , 1.9176502 ,
       0.        , 6.        , 0.        , 3.3308868 , 0.        ,
       6.        , 6.        , 3.0424356 , 5.390935  , 6.        ,
       2.445207  , 2.7478762 , 6.        , 2.1133027 , 0.     